In [1]:
import pandas as pd
import numpy as np
import pymongo as pymongo
import json
import pprint

In [2]:
#Esta celda se utiliza para cargar la base del select file
from tkinter import*
# Create Tk root
root = Tk()
# Hide the main window
root.withdraw()
root.call('wm', 'attributes', '.', '-topmost', True)

from tkinter import filedialog

### Actividades previas 

In [3]:
#Establece conexion 1)nombre de la base datos
client = pymongo.MongoClient("idenmon.zapto.org", 888)
db = client.prodLaboratorio
pruebas = db.MasterDiagnostico

In [4]:
pruebas_db = pruebas.find()
pruebas_list = list (pruebas_db)
row_preinsert = len(pruebas_list)

#### Generar un array de los codigos ya en la BD

### Leer archivo excel 

In [5]:
#Selecciona el archivo a cargar "master diagnostico suero.xlsx"
infiles = filedialog.askopenfilename(multiple=True)

%gui tk

In [6]:
#Cargar archivo de muestras
headers = ['id','qr','Lab_E6','Lab_CF','Lab_RV','Lab_VIH','App_E6','App_CF','App_RV','App_P24','Comp_E6','Comp_CF','Comp_RV','Comp_P24']
df = pd.ExcelFile(infiles[0])
df1 = df.parse('diagnostico individual')
df1.head()

,CODIGO,ESAT6,CFP10,RV1681,P24
0,101-VP,N,N,N,P
1,105-VP,N,N,N,P
2,107-VP,P,P,P,P
3,108-VP,N,N,N,P
4,109-VP,N,N,N,P


#### Eliminar las filas del DF que corrsponden a los codigos que ya tengo en la BD

### Validar Nombres de columnas 

In [7]:
#Validar nombres de columnas para carga a MongoDB
#col = list(df1)

In [8]:
#l= []
#for n in col:
   # l.append(n.lower().replace(" ", "_"))
#l    

In [9]:
#df1.columns = l
#df1.head()

In [10]:
codigo=[]
for n in range(0,len(pruebas_list)):
    codigo.append(pruebas_list[n]['CODIGO'])

In [11]:
dfRed = pd.DataFrame
dfRed = df1.drop(df1[df1['CODIGO'].isin(codigo)].index)
dfRed

,CODIGO,ESAT6,CFP10,RV1681,P24


### Convertir a JSON para poder almacenar en mongoDB

In [12]:
#Utilizando conversion directa a una lista de diccionarios en python
s = dfRed.to_json(orient = 'records')
l = json.loads(s)

### Guardar archivos en mongoDB

In [13]:
#Contar numero de registros con un checksum
num_rows_input = len (l)
num_rows_input

0

In [14]:
#Insertar JSON a mongoDB
if l:
    pruebas.insert_many(l)
else:
    print('No hay datos para guardar')

No hay datos para guardar


In [15]:
#desplegar en python df
pruebas_db = pruebas.find()
pruebas_list = list (pruebas_db)
dfp = pd.DataFrame(pruebas_list)




In [16]:
#Confirmar escritura
#num_rows_input
num_rows_output = len (pruebas_list)
#num_rows_output
sum_rows_input = (row_preinsert + num_rows_input)
ok_flag = (num_rows_output == sum_rows_input)
#ok flag
#num_rows_output
ok_flag

True